<p style="align: center;">
    <img align=center src="../img/dls_logo.jpg" width=500 height=500>
</p>

<h1 style="text-align: center;">
    Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ
</h1>

---

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline

Сгенерируем датасет "Игрушка Дьявола":

In [ ]:
# код для генерации взят из Стэнфордсокго курса:
# http://cs231n.github.io/neural-networks-case-study/#linear

N = 100
D = 2
K = 3
X = np.zeros((N * K, D))
y = np.zeros(N * K, dtype='uint8')

for j in range(K):
    ix = range(N * j, N * (j + 1))
    r = np.linspace(0, 1, N)
    t = np.linspace(j * 4, (j + 1) * 4, N) + np.random.randn(N) * 0.2 # theta
    X[ix] = np.c_[r * np.sin(t), r * np.cos(t)]
    y[ix] = j

In [ ]:
# отрисовочная магия, снова взято из
# http://cs231n.github.io/neural-networks-case-study/#linear

plt.figure(figsize=(10, 8))

plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.rainbow)

plt.title('Игрушка дьявола', fontsize=15)
plt.xlabel('$x$', fontsize=14)
plt.ylabel('$y$', fontsize=14)
plt.show()

In [ ]:
X = torch.FloatTensor(X)
y = torch.LongTensor(y)

Сейчас мы хотим научиться самостоятельно создавать наследников `nn.Module`. До этого мы делали нейросети с помощью класса `nn.Sequential`, попробуем построить такую же сеть, как на прошлом семинаре, но самостоятельно.

In [ ]:
# сеть с прошлого семинара

# D_in - размерность входа (количество признаков у объекта)
# H - размерность скрытых слоёв
# D_out - размерность выходного слоя (количество классов)
D_in, H, D_out = 2, 100, 3

# use the nn package to define our model and loss function
two_layer_net = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
    torch.nn.Softmax()
)

Что такое модуль и как он устроен? Во-первых, модуль это такой строительный блок для нейронок, с помощью модуля можно задать любую дифференциируемую по своему параметру функцию. Применяются модули так же, как и обычные функции с синтаксисом

```python
module_instance(var1, var2)
```

При этом внутри вызывается функция `forward` с теми же аргументами, а её выход возвращается как результат вызова модуля. Зачем же нужно так странно оборачивать обычные функции в модули? 

Это позволяет очень удобно следить за параметрами, которые надо изменять. Когда мы хотим получить все параметры, можно просто рекурсивно пройтись по всем полям модели, посмотреть, какие из них являются параметрами сами по себе, а какие являются модулями и содержат параметры внутри, а потом всё это объединить.

По этой причине, если вы используете внутри своего модуля какие-то ещё модули, их надо класть просто в поле класса, если это единичный модуль, и в класс **nn.ModuleList** или **nn.ModuleDict**, если у вас список или словарь используемых модулей. Если же в модели у вас есть какой-то собственный вес, то недостаточно положить тензор в поле класса, его надо обернуть в **nn.Parameter**, **nn.ParameterList** или **nn.ParameterDict** в зависимотси от того, какой именно у вас случай.

Такая организация позволяет достаточно безболезненно расширять **PyTorch** и писать для него свои функции, которые нельзя выразить композицией уже существующих. Пригождается это редко, поэтому сегодня мы не будем писать свое расширение.

Код, разделенный на модули - это просто красиво.

In [ ]:
# новая сеть

class MyModule(nn.Module):
    def __init__(self):
        super().__init__()
        self.f_lin = nn.Linear(D_in, H)
        self.s_lin = nn.Linear(H, D_out)
        
    def forward(self, X):
        X = F.relu(self.f_lin(X))
        return F.softmax(self.s_lin(X), dim=1)
    
model = MyModule()
loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), 1e-1)

Поговорим поподробнее о `softmax` и `CrossEntropyLoss`.

Напоминание: softmax-функция выглядит следующим образом:

$$
Softmax(x) =
\begin{pmatrix} 
    \dfrac{e^{x_1}}{\sum\limits_{i=0}^{m} e^{x_i}},\; \dots,\; \dfrac{e^{x_m}}{\sum\limits_{i=0}^{m} e^{x_i}}
\end{pmatrix}
$$

Таким образом, после применения softmax-функции мы получили вектор чисел из интервала $(0, 1)$, которые мы будем интерпретировать как вероятности классов.

Аналогично тому, как мы обощили сигмоиду на многоклассовый случай и получили softmax, можно обобщить и функцию потерь, получив **кросс-энтропию**:

$$
CrossEntropy(x) = -\sum\limits_{i=0}^m y_i \cdot log(Softmax(x)_i)
$$

где $y_i$ – истинная метка класса ($1$ если $x$ принадлежит $i$-ому классу и $0$ иначе).

In [ ]:
losses = []

for t in range(100):
    # forward
    y_pred = model(X)

    # loss
    loss = loss_fn(y_pred, y)
    losses.append(loss.item())

    optim.zero_grad()
    loss.backward()
    optim.step()
    
plt.plot(losses)
plt.show()

Теперь посмотрим, что будет, если не положить используемые внутри слои в `self`:

In [ ]:
# сеть без параметров

class MyModule(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_layers = [nn.Linear(D_in, H), nn.Linear(H, D_out)]
        self.my_useless_bias = torch.ones(1, H, requires_grad=True)
        self.more_of_my_useless_biases = [
            torch.ones(1, H, requires_grad=True),
            torch.ones(1, H, requires_grad=True),
            torch.ones(1, H, requires_grad=True),
        ]
        
    def forward(self, X):
        X = F.relu(self.linear_layers[0](X))
        X += self.my_useless_bias
        return F.softmax(self.linear_layers[1](X), dim=1)
    
model = MyModule()
list(model.parameters())

Как мы и ожидали, параметров нет. Исправим это:

In [ ]:
# исправленная сеть с параметрами

class MyModule(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_layers = nn.ModuleList([nn.Linear(D_in, H), nn.Linear(H, D_out)])
        self.my_useless_bias = nn.Parameter(torch.ones(1, H, requires_grad=True))
        self.more_of_my_useless_biases = nn.ParameterList([
            nn.Parameter(torch.ones(1, H, requires_grad=True)),
            nn.Parameter(torch.ones(1, H, requires_grad=True)),
            nn.Parameter(torch.ones(1, H, requires_grad=True))
        ])
        
    def forward(self, X):
        X = F.relu(self.linear_layers[0](X))
        X += self.my_useless_bias
        for b in self.more_of_my_useless_biases:
            X += b
        return F.softmax(self.linear_layers[1](X), dim=1)
    
model = MyModule()
list(model.parameters())

Как мы и хотели, все параметры появились!